# CS155 Project 3 Recurrent Neural Networks
In this notebook, we generate Shakespeare poems using LSTM-RNNs.

## Functions

In [138]:
import re
import pandas as pd
from keras.models import load_model
import WilliamRNN as willr
from nltk.tokenize import SyllableTokenizer

Ts = [1.5,0.75,0.25]
Nunits = [100,150,200]

def LoadRNNs(Nsemi):
    iT = 0
    models = []
    for Nunit in Nunits:
        model_Ts = []
        iT = 0
        for T in Ts:
            model = load_model('models/Nsemi'+str(Nsemi)+'/'+'rnn_iT_'+str(iT)+'_units_'+str(Nunit)+'.h5')
            iT += 1
            model_Ts.append(model)
        models.append(model_Ts)
    return models

def GetSyllablesDict():
    df = pd.read_csv('data/Syllable_dictionary.txt',header=None)
    df_vals = df.values
    syl_dic = {}
    syl_dic_E = {}
    for val in df_vals:
        dics = val[0].split()
        if len(dics) == 2:
            syl_dic[re.sub(r'[^\w]','',dics[0])] = int(dics[1])
        elif len(dics) == 3:
            try:
                syl_dic[re.sub(r'[^\w]','',dics[0])] = int(dics[2])
                eIDX = 1
            except:
                syl_dic[re.sub(r'[^\w]','',dics[0])] = int(dics[1])
                eIDX = 2
            syl_dic_E[re.sub(r'[^\w]','',dics[0])] = int(dics[eIDX].strip('E'))
    return syl_dic,syl_dic_E

def GetPoem(models,Ns):
    SSP = SyllableTokenizer()
    will = willr.William(seq_length=41,Nsemi=Ns)
    syl_dic,syl_dic_E = GetSyllablesDict()
    for i in range(len(Nunits)):
        for j in range(len(Ts)):
            model = models[i][j]
            print('####### LSTM-RNN: semi-redundant sequences with n = '+str(Ns)+' #######')
            print('T = '+str(Ts[j])+' and Nunit = '+str(Nunits[i]))
            input_c = list("shall i compare thee to a summer's day?\n")
            text = will.GenerateAIpoem(input_c,model,1000)
            lines = [line for line in text.split('\n')]
            Nsyl_list = []
            for l in range(len(lines)):
                line = lines[l]
                Nsyl = 0
                line_s = line.split()
                for word in line_s:
                    word = re.sub(r'[^\w]','',word).lower()
                    try:
                        if i != len(line):
                            Nsyl += syl_dic[word]
                        else:
                            Nsyl += syl_dic_E[word]
                    except:
                        if word != '':
                            Nsyl += len(SSP.tokenize(word))
                if Nsyl <= 9:
                    print('(#syllables  '+str(Nsyl)+')',line)
                else:
                    print('(#syllables '+str(Nsyl)+')',line)
                Nsyl_list.append(Nsyl)
                if l == 13:
                    break
            mean = sum(Nsyl_list)/len(Nsyl_list) 
            variance = sum([((x-mean) ** 2) for x in Nsyl_list])/len(Nsyl_list)
            std = variance ** 0.5
            print('Average #syllables: '+str(mean))
            print('Standard deviation #syllables: '+str(std))
            print('#############################################################','\n')
    pass
            
def GetErrors(models,Ns):
    will = willr.William(seq_length=41,Nsemi=Ns)
    for i in range(len(Nunits)):
        for j in range(len(Ts)):
            model = models[i][j]
            print('####### LSTM-RNN: semi-redundant sequences with n = '+str(Ns)+' #######')
            print('T = '+str(Ts[j])+' and Nunit = '+str(Nunits[i]))
            model.fit(will.X,will.y,epochs=1,verbose=2)
            print('#############################################################')
    pass

def GetSinglePoem(model,input_c,Ns):
    SSP = SyllableTokenizer()
    will = willr.William(seq_length=41,Nsemi=Ns)
    syl_dic,syl_dic_E = GetSyllablesDict()
    print('############### My favorite poem ###############')
    text = will.GenerateAIpoem(input_c,model,1000)
    lines = [line for line in text.split('\n')]
    Nsyl_list = []
    for l in range(len(lines)):
        line = lines[l]
        Nsyl = 0
        line_s = line.split()
        for word in line_s:
            word = re.sub(r'[^\w]','',word).lower()
            try:
                if i != len(line):
                            Nsyl += syl_dic[word]
                else:
                            Nsyl += syl_dic_E[word]
            except:
                if word != '':
                    Nsyl += len(SSP.tokenize(word))
        if Nsyl <= 9:
            print('(#syllables  '+str(Nsyl)+')',line)
        else:
            print('(#syllables '+str(Nsyl)+')',line)
        Nsyl_list.append(Nsyl)
        if l == 13:
            break
    mean = sum(Nsyl_list)/len(Nsyl_list) 
    variance = sum([((x-mean) ** 2) for x in Nsyl_list])/len(Nsyl_list)
    std = variance ** 0.5
    print('Average #syllables: '+str(mean))
    print('Standard deviation #syllables: '+str(std))
    print('#################################################','\n')
    pass

## Load LSTM-RNNs (trained with semi-redundant sequences with n = 5, 10)

In [139]:
models1 = LoadRNNs(1)
models5 = LoadRNNs(5)
models10 = LoadRNNs(10)

## Generate poems using LSTM-RNNs

In [141]:
GetPoem(models1,1)

####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 1.5 and Nunit = 100
(#syllables 10) shall i compare thee to a summer's day?
(#syllables 15) when thou shall summer's that which the world so de.' when the satf's spent.
(#syllables 12)   to bution bear detime to him out of true,
(#syllables 10) and the world's from the sumper to the sake,
(#syllables 10) and then those all the worls more than the dear.
(#syllables 10) which where i have so strangle as that seen,
(#syllables 11) and theneir merally mustant warthing bear
(#syllables  9) which now are my love to the sides kear
(#syllables  9) with the world's from the to mour would may.
(#syllables  9)   the swell of thy love and love's frown stand
(#syllables 11)   to kingers maded a contross that thou art,
(#syllables 10)   and the world with to the sabs of spatce.
(#syllables 13) it self the worsh to show the world woul slave some for least,
(#syllables 10) when thou form of thy love and love and see,
Average #syllables:

(#syllables 10) shall i compare thee to a summer's day?
(#syllables 10) thou art more lovely and more temperate:
(#syllables 12) rough winds doside his tracariess of at fies,
(#syllables 10) whose stall grace in warth of thine, her create,
(#syllables 10) that have i not locked up in any chest,
(#syllables 10) save where thou art not, though i feel thou art,
(#syllables 10) within the gentle closure of my breast,
(#syllables 11) for being vurse in this good to this shape,
(#syllables 11) whilst he is, tellive growin that loving,
(#syllables 11) peins how there be beloved as a will me,
(#syllables 11)   birdren outare much is canker mine eye,
(#syllables 12) as i not fave, or sounded sickly is out,
(#syllables 10)   and they as farth time's bectuse do nend,
(#syllables 10) than of your graces and your gifts to tell.
Average #syllables: 10.571428571428571
Standard deviation #syllables: 0.7284313590846835
############################################################# 



In [111]:
GetPoem(models5,5)

####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 1.5 and Nunit = 100
(#syllables 10) shall i compare thee to a summer's day?
(#syllables 11) for whiness that is mert love when as whate,
(#syllables  8) and remoch from thee frost have worth,
(#syllables 10) and may you lepptes with hath that i low,
(#syllables 16) nay what see thou death fort mine wairiny beautyoy beauty,
(#syllables  9) that that liss gle for see that frim might?
(#syllables 12) doth my body day my orlons of though sweetes,
(#syllables  9) that tour wells of thee dord eyes say lay,
(#syllables 11) that lend then in enore thou hemmings mine,
(#syllables  7) and this for that wheredy,
(#syllables 11) and trisur surt tingur waired wild thy prams,
(#syllables 13) at this worder that my lofor thou vertured shand,
(#syllables 10) since me that is meater'se of thy see,
(#syllables 10) and look in a love, the it love in net,
Average #syllables: 10.5
Standard deviation #syllables: 2.1297216451250818
###########

(#syllables 10) shall i compare thee to a summer's day?
(#syllables 16) nor that the cames and evesure to ame worth nove,
(#syllables 11)   for the time upon call i love that sare,
(#syllables 11) as vend that con tell mume spenks ounder art.
(#syllables 11) the fairy of that sair storn othe surmer,
(#syllables  8) and prach therefore is thy beepent,
(#syllables 10) when your seef ounds ad im dost prof mempers
(#syllables 14)   where thee bet eys ridesereceed of your freasured,
(#syllables 10)   thoughthence sammer drow stounded of trim,
(#syllables  8) nor ampering of thee dost leave,
(#syllables 11) up in thy belf beth restall mume bringlest,
(#syllables 12)   then all the reser-subut of thines rown steent.
(#syllables 12)   wild thy bedow i confer each do his sine,
(#syllables  9) ear it thus roustion yes richtere:
Average #syllables: 10.928571428571429
Standard deviation #syllables: 2.086154552358605
############################################################# 



In [112]:
GetPoem(models10,10)

####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 1.5 and Nunit = 100
(#syllables 10) shall i compare thee to a summer's day?
(#syllables 10) whan thou art buthtesull be tough mery,
(#syllables 14) no fouldy premen eabtedo not fan rome tous groegh,
(#syllables 14) for with mumy lo seepich they reme to tee sume,
(#syllables 10) and your nee focke tress a vill noth live,
(#syllables  3)   leabey look,
(#syllables 15) ond not tat in i ne rind, and seeft suclift cate sime,
(#syllables  3) and your nee,
(#syllables 12) soise sace and suemen enouchich then frown,
(#syllables 10) when well fay folln or mand and ther foren,
(#syllables  8) that which stool deather ene lays,
(#syllables 13) and searing thit woyns tide's ove ar time is fore.
(#syllables  5)   in me sel sine,
(#syllables 10) soresting thou do spilens in ton end,
Average #syllables: 9.785714285714286
Standard deviation #syllables: 3.7450647796909515
############################################################# 

#

(#syllables 10) shall i compare thee to a summer's day?
(#syllables 12) of that whor and co me the lives  is doresse,
(#syllables 11) and deting houn ail thy welfsh mock on elle,
(#syllables 11) bud i (houre orthengeded blatker brong.
(#syllables  1)   fress,
(#syllables 13) ind budond freem is an, and ey she thee do doth rear,
(#syllables 12) han had uthat the stmmerewghtoof llle my praite,
(#syllables  6)  hich shor tobe so wrong;
(#syllables 12) doy you ir hill to the lover woth wheesest beak,
(#syllables 10) or mer me ir thearess'aiker try sind
(#syllables 12) tore art or thy sinf livv's w mond saight fros mo,
(#syllables 10) in the piringn dand wor doy betull pith,
(#syllables 13) that cow thou well be upleing of and thy mroughtiof sree:
(#syllables 10) sed aod then memmer plothth tho lov'st commend,
Average #syllables: 10.214285714285714
Standard deviation #syllables: 3.0747490357186975
############################################################# 



## Obtain accuracies of each LSTM-RNN

In [142]:
GetErrors(models1,1)

####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 1.5 and Nunit = 100
Epoch 1/1
 - 69s - loss: 1.0131 - accuracy: 0.6738
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 0.75 and Nunit = 100
Epoch 1/1
 - 77s - loss: 0.8428 - accuracy: 0.7230
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 0.25 and Nunit = 100
Epoch 1/1
 - 87s - loss: 0.7011 - accuracy: 0.7655
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 1.5 and Nunit = 150
Epoch 1/1
 - 116s - loss: 0.9248 - accuracy: 0.7009
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 1 #######
T = 0.75 and Nunit = 150
Epoch 1/1
 - 114s - loss: 0.3428 - accuracy: 0.8800
#############################################################


In [105]:
GetErrors(models5,5)

####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 1.5 and Nunit = 100
Epoch 1/1
 - 35s - loss: 0.1992 - accuracy: 0.9450
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 0.75 and Nunit = 100
Epoch 1/1
 - 36s - loss: 0.0928 - accuracy: 0.9760
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 0.25 and Nunit = 100
Epoch 1/1
 - 36s - loss: 0.0078 - accuracy: 0.9999
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 1.5 and Nunit = 150
Epoch 1/1
 - 44s - loss: 0.1805 - accuracy: 0.9515
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 5 #######
T = 0.75 and Nunit = 150
Epoch 1/1
 - 45s - loss: 0.0913 - accuracy: 0.9822
#############################################################
##

In [107]:
GetErrors(models10,10)

####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 1.5 and Nunit = 100
Epoch 1/1
 - 19s - loss: 0.1596 - accuracy: 0.9660
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 0.75 and Nunit = 100
Epoch 1/1
 - 18s - loss: 0.0339 - accuracy: 0.9999
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 0.25 and Nunit = 100
Epoch 1/1
 - 18s - loss: 0.0039 - accuracy: 1.0000
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 1.5 and Nunit = 150
Epoch 1/1
 - 23s - loss: 0.0167 - accuracy: 1.0000
#############################################################
####### LSTM-RNN: semi-redundant sequences with n = 10 #######
T = 0.75 and Nunit = 150
Epoch 1/1
 - 23s - loss: 0.0053 - accuracy: 1.0000
###########################################################

## My favorite poem

In [157]:
GetSinglePoem(models1[2][1],list("i can't stop this feeling, deep inside of me,\n"),1)

############### My favorite poem ###############
(#syllables 12) i can't stop this feeling, deep inside of me,
(#syllables 13) liquing in thine eye bate my gross do bent,
(#syllables 11) by advich controw it so forgot a lick,
(#syllables 14) i may begues, that thou mayst have those their dear deem,
(#syllables 12) but what of thee, and for thy self art pleasure,
(#syllables 12)   but daves more blest thee vanticing crossed,
(#syllables 12) the poor of this proud come his rick) compite!
(#syllables  9) hoon, thou the sweetest broods on thee it;
(#syllables 13) both from thy fair) to flatterer thou abundarness,
(#syllables 11) right despised strangle and love doth seen,
(#syllables 12) all stars in thy heart is love therefore,
(#syllables 12)   they may detiven makes and soonur all wrong.
(#syllables 13) i will not polis conforned af your rarse,
(#syllables 14) their hold and then more in hope's all, with men's eyes,
Average #syllables: 12.142857142857142
Standard deviation #syllables: 1